In [27]:
import pandas as pd
import numpy as np
from datetime import datetime

In [21]:
# 1번 함수 생성
def create_band(df, col):
    # 결측치와 이상치를 제가
    df = df.loc[~df.isin((np.nan, np.inf, -np.inf)).any(axis='columns'), [col]]
    # 수정 종가 컬럼을 제외한 데이터프레임 생성
    # df = df[['Adj Close']]

    # 이동 평균선 생성
    df['center'] = df.rolling(20).mean()

    # 상단 밴드 생성
    df['ub'] = df['center'] + ( 2 * df[col].rolling(20).std() )
    # 하단 밴드 생성
    df['lb'] = df['center'] - ( 2 * df[col].rolling(20).std() )
    # 결과를 리턴
    return df

In [13]:
# 2번 함수 생성
def add_trade(df):
    # trade 파생변수 생성
    df['trade'] = ""

    # 거래 내역을 추가
    for i in df.index:
    # 상단 밴드보다 종가가 높은 경우
        if df.loc[i, 'Adj Close'] > df.loc[i, 'ub']:
            # 현재 구매 상태이면
            if df.shift(1).loc[i, 'trade'] == 'buy':
                # 매도
                df.loc[i, 'trade'] = ''
            else:
                df.loc[i, 'trade'] = ''
        # 하단 밴드보다 종가가 낮은 경우
        elif df.loc[i,'Adj Close'] < df.loc[i,'lb']:
            # 현재 구매 상태이면
            if df.shift(1).loc[i, 'trade'] == 'buy':
                # 구매 상태를 유지
                df.loc[i, 'trade'] = 'buy'
            else:
                df.loc[i, 'trade'] = 'buy'
        else:
            # 현재 구매 상태면
            if df.shift(1).loc[i, 'trade'] == 'buy':
                # 구매 상태를 유지
                df.loc[i, 'trade'] = 'buy'
            else:
                df.loc[i, 'trade'] = ''
    return df

In [18]:
# 3번 함수 생성
def add_rtn(df):
    # 수익률 파생변수 생성
    rtn = 1.0

    df['return'] = 1
    buy = 0.0
    sell = 0.0

    for i in df.index:
        # 구매가를 출력
        if (df.shift(1).loc[i, 'trade'] == '') and \
            (df.loc[i, 'trade'] == 'buy'):
            buy = df.loc[i, 'Adj Close']
            print('진입일 :', i, '구매 가격 :', buy)
        # 판매가를 출력
        elif (df.shift(1).loc[i, 'trade'] == 'buy') and \
            (df.loc[i, 'trade'] == ''):
            sell = df.loc[i, 'Adj Close']
            rtn = (sell - buy)/buy + 1
            df.loc[i, 'return'] = rtn
            print('판매일 :', i, '판매 가격 :', sell, '수익률 :', rtn)

        # 구매가, 판매가를 초기화
        if df.loc[i, 'trade'] == '':
            buy = 0.0
            sell = 0.0
    # 누적 수익률을 계산하여 파생변수에 대입
        acc_rtn = 1.0
        for i in df.index:
            rtn = df.loc[i, 'return']
            acc_rtn *= rtn
            df.loc[i, 'acc_rtn'] = acc_rtn
        return df

In [22]:
data = pd.read_csv("../csv/GM.csv", index_col='Date')

In [23]:
data = create_band(data, 'Adj Close')

In [24]:
data.iloc[18:25]

,Adj Close,center,ub,lb
Date,,,,
2010-12-15,26.568705,NaN,NaN,NaN
2010-12-16,26.568705,26.922061,27.622244,26.221877
2010-12-17,26.877001,26.914551,27.613207,26.215895
2010-12-20,26.687281,26.894789,27.595795,26.193783
2010-12-21,26.758423,26.885698,27.588934,26.182461
2010-12-22,27.604259,26.951704,27.664945,26.238464
2010-12-23,27.517311,27.004273,27.721733,26.286812


In [16]:
add_trade(data)['trade'].value_counts()

buy    1130
       1032
Name: trade, dtype: int64

In [17]:
data = add_trade(data)

In [19]:
add_rtn(data)

,Adj Close,center,ub,lb,trade,return,acc_rtn
Date,,,,,,,
2010-11-18,27.027195,NaN,NaN,NaN,,1,1.0
2010-11-19,27.082527,NaN,NaN,NaN,,1,1.0
2010-11-22,26.940243,NaN,NaN,NaN,,1,1.0
2010-11-23,26.284130,NaN,NaN,NaN,,1,1.0
2010-11-24,26.465942,NaN,NaN,NaN,,1,1.0
...,...,...,...,...,...,...,...
2019-06-18,36.700001,35.249369,37.239697,33.259041,buy,1,1.0
2019-06-19,36.779999,35.251553,37.248834,33.254271,buy,1,1.0
2019-06-20,36.959999,35.340899,37.478352,33.203445,buy,1,1.0


In [ ]:
import bollinger as bg
# 라이브러리 리로드
import imp
imp.reload(bg)

In [46]:
# 클래스 생성
data = pd.read_csv("../csv/AAPL.csv", index_col='Date')

In [47]:
class_a = bg.Invest(data, 'Adj Close')

In [48]:
class_a.create_band('20100101', '20150101')

,Adj Close,center,ub,lb
Date,,,,
2010-01-04,26.782711,25.037723,27.046734,23.028713
2010-01-05,26.829010,25.169503,27.288098,23.050908
2010-01-06,26.402260,25.307290,27.366449,23.248130
2010-01-07,26.353460,25.436879,27.410937,23.462821
2010-01-08,26.528664,25.525609,27.529742,23.521475
...,...,...,...,...
2014-12-24,103.617920,104.542070,110.388364,98.695775
2014-12-26,105.449570,104.310338,109.569622,99.051054
2014-12-29,105.375557,104.078143,108.639948,99.516339


In [49]:
class_a.df

,Adj Close,center,ub,lb
Date,,,,
2010-01-04,26.782711,25.037723,27.046734,23.028713
2010-01-05,26.829010,25.169503,27.288098,23.050908
2010-01-06,26.402260,25.307290,27.366449,23.248130
2010-01-07,26.353460,25.436879,27.410937,23.462821
2010-01-08,26.528664,25.525609,27.529742,23.521475
...,...,...,...,...
2014-12-24,103.617920,104.542070,110.388364,98.695775
2014-12-26,105.449570,104.310338,109.569622,99.051054
2014-12-29,105.375557,104.078143,108.639948,99.516339


In [50]:
class_a.add_trade()

,Adj Close,center,ub,lb,trade
Date,,,,,
2010-01-04,26.782711,25.037723,27.046734,23.028713,
2010-01-05,26.829010,25.169503,27.288098,23.050908,
2010-01-06,26.402260,25.307290,27.366449,23.248130,
2010-01-07,26.353460,25.436879,27.410937,23.462821,
2010-01-08,26.528664,25.525609,27.529742,23.521475,
...,...,...,...,...,...
2014-12-24,103.617920,104.542070,110.388364,98.695775,buy
2014-12-26,105.449570,104.310338,109.569622,99.051054,buy
2014-12-29,105.375557,104.078143,108.639948,99.516339,buy


In [51]:
class_a.df

,Adj Close,center,ub,lb,trade
Date,,,,,
2010-01-04,26.782711,25.037723,27.046734,23.028713,
2010-01-05,26.829010,25.169503,27.288098,23.050908,
2010-01-06,26.402260,25.307290,27.366449,23.248130,
2010-01-07,26.353460,25.436879,27.410937,23.462821,
2010-01-08,26.528664,25.525609,27.529742,23.521475,
...,...,...,...,...,...
2014-12-24,103.617920,104.542070,110.388364,98.695775,buy
2014-12-26,105.449570,104.310338,109.569622,99.051054,buy
2014-12-29,105.375557,104.078143,108.639948,99.516339,buy


In [52]:
class_a.add_rtn()

진입일 : 2010-01-22 00:00:00 구매 가격 : 24.747818
판매일 : 2010-03-01 00:00:00 판매 가격 : 26.154476 수익율 : 1.0568396777445188
진입일 : 2010-08-24 00:00:00 구매 가격 : 30.026524
판매일 : 2010-09-08 00:00:00 판매 가격 : 32.90366 수익율 : 1.0958198158401553
진입일 : 2011-03-16 00:00:00 구매 가격 : 41.299767
판매일 : 2011-07-01 00:00:00 판매 가격 : 42.957966 수익율 : 1.040150323366231
진입일 : 2011-11-14 00:00:00 구매 가격 : 47.463268
판매일 : 2011-12-27 00:00:00 판매 가격 : 50.876015 수익율 : 1.0719029081604747
진입일 : 2012-04-16 00:00:00 구매 가격 : 72.601524
판매일 : 2012-06-18 00:00:00 판매 가격 : 73.308609 수익율 : 1.0097392583659814
진입일 : 2012-10-08 00:00:00 구매 가격 : 80.207954
판매일 : 2013-07-29 00:00:00 판매 가격 : 57.243137 수익율 : 0.7136840443530077
진입일 : 2013-09-11 00:00:00 구매 가격 : 60.184383
판매일 : 2013-10-18 00:00:00 판매 가격 : 65.48336 수익율 : 1.0880457144505413
진입일 : 2014-01-03 00:00:00 구매 가격 : 70.019096
판매일 : 2014-03-25 00:00:00 판매 가격 : 70.960335 수익율 : 1.0134426042861222
진입일 : 2014-10-15 00:00:00 구매 가격 : 89.842468
판매일 : 2014-10-23 00:00:00 판매 가격 : 96.557182 수익율 : 1.074

In [53]:
class_a.df

,Adj Close,center,ub,lb,trade,return,acc_rtn
Date,,,,,,,
2010-01-04,26.782711,25.037723,27.046734,23.028713,,1.0,1.000000
2010-01-05,26.829010,25.169503,27.288098,23.050908,,1.0,1.000000
2010-01-06,26.402260,25.307290,27.366449,23.248130,,1.0,1.000000
2010-01-07,26.353460,25.436879,27.410937,23.462821,,1.0,1.000000
2010-01-08,26.528664,25.525609,27.529742,23.521475,,1.0,1.000000
...,...,...,...,...,...,...,...
2014-12-24,103.617920,104.542070,110.388364,98.695775,buy,1.0,1.102717
2014-12-26,105.449570,104.310338,109.569622,99.051054,buy,1.0,1.102717
2014-12-29,105.375557,104.078143,108.639948,99.516339,buy,1.0,1.102717
